In [1]:
# First import the packages we'll need
# Numpy is for numerical analysis
import numpy as np
# Pandas is for data storage and manipulation
import pandas as pd
# Matplotlib and seaborn are for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


1. Go to the following website: https://www.rug.nl/ggdc/productivity/pwt/

Download Penn World Table version 9.1 database in your preferred format. Consider the following countries: UK, USA, Denmark, France, and Colombia.

a.  Using data in 2017, calculate real output per worker, real capital stock per worker, and human capital index for each of the countries you picked. Use 'rgdpna' (Real GDP at constant 2011 national prices (in mil. 2011USD)) as a measure of real GDP, use 'rnna' (Capital stock at constant 2011 national prices (in mil. 2011USD)) as a measure of real capital stock, use 'emp'(Number of persons engaged (in millions)) as a measure of employment, and use 'hc' (Human capital index, based on years of schooling and returns to education) as a measure of human capital.

b. Assume a Cobb-Douglas production function with $\alpha=1/3$. Suppose countries differ only with respect to their investment rates. For each country listed above, calculate Solow model's predicted output per worker relative to the UK output per worker. For investment rates of countries, use average of variable csh\_i (Share of gross capital formation at current PPPs) over the years 1970-2017. Generate a table comparing Solow model's predicted output per worker relative to the UK with actual output per worker relative to the UK (in 2017, or in the latest available year). Hint, I ask you to do an exercise similar to what we have done in lecture 3. Briefly comment on the table.  

c. Now, assume that countries differ with respect to investment rates and employment growth rates. Assume that depreciation rates in all countries are equal to 5%, $\delta=0.05$. Calculate average annual employment growth rates of the above listed countries from 1970 to 2017. Repeat the exercise in part b. Create a table that compares Solow model's predicted output per worker differences with actual output per worker differences relative to the UK. Briefly comment on the table.

d. Now, assume that countries differ not only in their investment and employment growth rates but also in their human capital. For this exercise use variable hc in 2017 (Human capital index, based on years of schooling and returns to education) as $h$ in the Solow model. Assume that depreciation rates in all countries are equal to 5%, $\delta=0.05$. . Repeat part c while considering human capital differences across countries. Create a table and briefly comment on it.

e.Do your predictions approach to actual income differences as you take into account more variables? 



In [3]:
# Here, I download the Penn World Tables legend
df_legend = pd.read_excel('https://www.rug.nl/ggdc/docs/pwt100.xlsx',sheet_name='Legend')
df_legend.dropna(subset=['Variable name','Variable definition'],inplace=True)
# I create a dictionary of variables names and definitions
df_legend = dict(zip(df_legend['Variable name'],df_legend['Variable definition']))

In [5]:
# Here are the variables, we will need in this analysis
print('cgdpe = ', df_legend['cgdpe'])
print('rnna = ', df_legend['rnna'])
print('emp = ', df_legend['emp'])
print('hc =', df_legend['hc'])
print('csh_i =', df_legend['csh_i'])

cgdpe =  Expenditure-side real GDP at current PPPs (in mil. 2017US$)
rnna =  Capital stock at constant 2017 national prices (in mil. 2017US$)
emp =  Number of persons engaged (in millions)
hc = Human capital index, based on years of schooling and returns to education; see Human capital in PWT9.
csh_i = Share of gross capital formation at current PPPs


In [6]:
# now download the data
df = pd.read_excel('https://www.rug.nl/ggdc/docs/pwt100.xlsx',sheet_name='Data')

In [7]:
# create GDP per worker 
df['gdpPerworker'] = df['cgdpe']/df.emp

In [8]:
# Penn World Tables contain many variables, but these are the variables ...
# I need to create the table asked in part a
var_list = ['countrycode','country','cgdpe','emp','hc','year']

In [9]:
## Here, we create the table asked in part a)

# This cell does five things:
# 1) It selects the countries in my list: df.countrycode.isin(country_list)
# 2) It selects year 2017: (df.year==2017)
# 3) It selects the variables that I want to display :
#               [['country','year','gdpPerworker','capPerworker','hc','emp','pop']]
# 4) It rounds the variables to 2 decimal points: .round(decimals=2)
# 5) Sets country as index: .set_index('country') 
df[(df.year==2019)][
    ['country','year','gdpPerworker','hc','emp','pop']].round(decimals=2)

,country,year,gdpPerworker,hc,emp,pop
69,Aruba,2019,82190.37,NaN,0.05,0.11
139,Angola,2019,13684.12,1.48,16.64,31.83
209,Anguilla,2019,NaN,NaN,NaN,0.01
279,Albania,2019,33282.29,2.96,1.08,2.88
349,United Arab Emirates,2019,116760.30,2.75,5.81,9.77
...,...,...,...,...,...,...
12529,Viet Nam,2019,14838.50,2.87,50.40,96.46
12599,Yemen,2019,9027.14,1.84,5.53,29.16
12669,South Africa,2019,40136.11,2.91,18.64,58.56
12739,Zambia,2019,11041.17,2.69,5.23,17.86


In [10]:
## To solve part b, I first need to calculate average investment rate ...
# for each country from 1970 to 2007

# This cell does the following things:
# 1) It selects data from years 1970 to 2017: df[df.year.isin(np.arange(1970,2018))]
# np.arange(1970,2018) creates a list from 1970 to 2017.
# intervals on python is right-open. Hence np.arange(1970,2018) creates an array that does not inclue 2018
# 2) it takes averages of the investment rates for each country:
                #.groupby(['countrycode','country'])['csh_i'].mean()
# 3) it resets data index
# 4) it renames 'csh_i' variable to "Investment rate"
    
begin,end=1990,2019
df_fundamentals = (df[df.year.isin(np.arange(begin,end))]
                   .groupby(['countrycode','country'])['csh_i'].mean().reset_index().rename(
                       columns={'csh_i':'Investment rate'}))

In [11]:
# Now look at he investment rate of the countries we calculated above
# In fact we calculated invesment rate for each country in our data
# we display investment rate only for the countries we are interested in

df_fundamentals.set_index('countrycode',inplace=True)
df_fundamentals.round(decimals=2)

,country,Investment rate
countrycode,,
ABW,Aruba,0.36
AGO,Angola,0.37
AIA,Anguilla,0.49
ALB,Albania,0.19
ARE,United Arab Emirates,0.30
...,...,...
VNM,Viet Nam,0.21
YEM,Yemen,0.12
ZAF,South Africa,0.17


In [12]:
# In part c, we need to calculate the average employment growth

In [13]:
# first sort our data by country and by year
# we need by year sorting to ensure that 1970 data comes earlier than 2017 data ...
# in our dataset. We need this in the next cell
df.sort_values(['countrycode','year'],inplace=True)

In [14]:
# This cell calculates the average annual employment growth rate from 1970 to 2017
# It first selects the years 1970 and 2017
# For each country it calculate average annual employment growth rate ...
# using this formula = (emp_2017/emp_1970)^(1/47)-1
# there are multiple ways of calculating average employment growth rate,...
# the above is one of them

# Here is how code works:
# 1) it selects data from years 1970 and 2017: df[df.year.isin([1970,2017])]
# 2) It groups the data based on countrycodes: .groupby(['countrycode']
# 3) for each country code, we have 2 observations, from 1970 and from 2017
# x['emp'].values[0] is the first observation from 1970 
# x['emp'].values[1] is the second observation from 2017
# the average annual employment growth is equal to x['emp'].values[1]/x['emp'].values[0])**(1/47)-1, ...
# where x represent a country
# 4) Lastly it renames what we calculated as 'Employment growth': .rename(columns={0:'Employment growth'})

df_emp_growth = (df[df.year.isin([begin,end])]
                 .groupby(['countrycode']).apply(lambda x: (x['emp'].values[1]/x['emp'].values[0])**(1/(end-begin))-1)
                 .reset_index().rename(columns={0:'Employment growth'}))

In [15]:
# show the employment growth rate for the countries we selected
df_emp_growth.set_index('countrycode',inplace=True)
df_emp_growth.round(decimals=3)

,Employment growth
countrycode,
ABW,NaN
AGO,0.032
AIA,NaN
ALB,-0.007
ARE,0.064
...,...
VNM,0.019
YEM,0.033
ZAF,0.016


In [16]:
# merge df_fundamentals data (contains invesment rates) with df_emp_growth (contains employment growth rate)
df_fundamentals= df_fundamentals.join(df_emp_growth,how='left')

In [17]:
df

,countrycode,country,currency_unit,year,rgdpe,rgdpo,pop,emp,avh,hc,...,csh_m,csh_r,pl_c,pl_i,pl_g,pl_x,pl_m,pl_n,pl_k,gdpPerworker
0,ABW,Aruba,Aruban Guilder,1950,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABW,Aruba,Aruban Guilder,1951,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABW,Aruba,Aruban Guilder,1952,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,Aruban Guilder,1953,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABW,Aruba,Aruban Guilder,1954,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12805,ZWE,Zimbabwe,US Dollar,2015,40141.617188,39798.644531,13.814629,6.393752,NaN,2.584653,...,-0.287693,-0.051930,0.479228,0.651287,0.541446,0.616689,0.533235,0.422764,1.534175,6250.043014
12806,ZWE,Zimbabwe,US Dollar,2016,41875.203125,40963.191406,14.030331,6.504374,NaN,2.616257,...,-0.251232,-0.016258,0.470640,0.651027,0.539631,0.619789,0.519718,0.416510,1.492129,6408.133575
12807,ZWE,Zimbabwe,US Dollar,2017,44672.175781,44316.742188,14.236595,6.611773,NaN,2.648248,...,-0.202827,-0.039897,0.473560,0.639560,0.519956,0.619739,0.552042,0.415592,1.515128,6756.459377
12808,ZWE,Zimbabwe,US Dollar,2018,44325.109375,43420.898438,14.438802,6.714952,NaN,2.680630,...,-0.263658,-0.020791,0.543757,0.655473,0.529867,0.641361,0.561526,0.425143,1.590753,6603.539858


In [18]:
# in part d, we will need human capital values
# merge our df_fundamentals data with the Penn World Tables
# but, we don't need all of PWT, select only year 2017, and other required variables
df_fundamentals = df_fundamentals.join(df[df.year == end].set_index('countrycode')[
    ['emp','gdpPerworker','hc']],how='left')

In [32]:
print(df_fundamentals.loc[['DEU','MLI']][['Investment rate','Employment growth','hc']
                                        ].transpose().round(decimals=3).to_latex())

\begin{tabular}{lrr}
\toprule
countrycode &    DEU &    MLI \\
\midrule
Investment rate   &  0.237 &  0.142 \\
Employment growth &  0.004 &  0.031 \\
hc                &  3.675 &  1.380 \\
\bottomrule
\end{tabular}



In [ ]:
# we will calculate Solow's predicted output per worker relative to the UK
# hence, create a different data just for the UK values
gbr = df_fundamentals.loc['GBR']

In [24]:
# now add columns to df_fundamentals data, consisting of corresponding values from the UK
for var in ['Investment rate','Employment growth','hc','gdpPerworker']:
    df_fundamentals[f'{var}, GBR'] = gbr[var]

In [27]:
# here is our data
df_fundamentals.round(decimals=3)

,country,Investment rate,Employment growth,emp,gdpPerworker,hc,"Investment rate, GBR","Employment growth, GBR","hc, GBR","gdpPerworker, GBR"
countrycode,,,,,,,,,,
ABW,Aruba,0.358,NaN,0.048,82190.373,NaN,0.233,0.007,3.774,94307.441
AGO,Angola,0.371,0.032,16.645,13684.117,1.482,0.233,0.007,3.774,94307.441
AIA,Anguilla,0.488,NaN,NaN,NaN,NaN,0.233,0.007,3.774,94307.441
ALB,Albania,0.191,-0.007,1.076,33282.294,2.965,0.233,0.007,3.774,94307.441
ARE,United Arab Emirates,0.297,0.064,5.809,116760.296,2.747,0.233,0.007,3.774,94307.441
...,...,...,...,...,...,...,...,...,...,...
VNM,Viet Nam,0.212,0.019,50.400,14838.497,2.870,0.233,0.007,3.774,94307.441
YEM,Yemen,0.125,0.033,5.532,9027.143,1.843,0.233,0.007,3.774,94307.441
ZAF,South Africa,0.172,0.016,18.643,40136.115,2.908,0.233,0.007,3.774,94307.441


In [28]:
# We have prepared our data to conduct the required analysis
# First set our parameter values
alpha=1/3
delta = .05
# In many parts of the analysis, we will write alpha/(1-alpha)...
# create a new variable to redue typing
alpham = alpha/(1-alpha)

Notice that at the steady state of the Solow model income per worker is equal to
$$y^\ast = A^{1/(1-\alpha)}\left(\frac{\gamma}{\delta+n} \right)^{\alpha/(1-\alpha)}h.$$

Therefore income per worker ratios are (assuming countries have the same technology, $A$):
$$\frac{y_i}{y_{UK}} = \left(\frac{\frac{\gamma_i}{\delta+n_i}}{\frac{\gamma_{UK}}{\delta+n_{UK}}} \right)^{\alpha/(1-\alpha)}\frac{h_i}{h_{UK}}.$$

Rewrite the above formula:
$$\frac{y_i}{y_{UK}} = \left(\frac{\gamma_i}{\gamma_{UK}}\right)^{\alpha/(1-\alpha)}\left(\frac{\delta+n_{UK}}{\delta+n_i}\right)^{\alpha/(1-\alpha)}\frac{h_i}{h_{UK}}.$$




In part b, we assume countries differ only with respect to their investment rates. Therefore:
$$\frac{y_i}{y_{UK}} = \left(\frac{\gamma_i}{\gamma_{UK}}\right)^{\alpha/(1-\alpha)}.$$

    


In [29]:
# calculate Solow's predicted income per worker ratios as in the above formula
df_fundamentals['rel_GDP_pred_inv']=(df_fundamentals['Investment rate']/df_fundamentals['Investment rate, GBR'])**alpham

In [30]:
# calculate the actual output per worker rations from data
df_fundamentals['rel_GDP'] = df_fundamentals['gdpPerworker']/df_fundamentals['gdpPerworker, GBR']

In part c, countries differ with respect to their employment growth rates as well as their investment rates:

$$\frac{y_i}{y_{UK}} = \left(\frac{\frac{\gamma_i}{\delta+n_i}}{\frac{\gamma_{UK}}{\delta+n_{UK}}} \right)^{\alpha/(1-\alpha)}.$$




In [31]:
## calculate Solow's predicted income per worker ratios as in the above formula
df_fundamentals['rel_GDP_pred_inv_emp']=((df_fundamentals['Investment rate']/
                                          (delta+df_fundamentals['Employment growth']))
                                         /(df_fundamentals['Investment rate, GBR']/
                                          (delta+df_fundamentals['Employment growth, GBR'])))**alpham

In part d, countries differ with respect to their human capital, employment growth rate and investment rate



$$\frac{y_i}{y_{UK}} = \left(\frac{\frac{\gamma_i}{\delta+n_i}}{\frac{\gamma_{UK}}{\delta+n_{UK}}} \right)^{\alpha/(1-\alpha)}\frac{h_i}{h_{UK}}.$$



In [32]:
## calculate Solow's predicted income per worker ratios as in the above formula
df_fundamentals['rel_GDP_pred_inv_emp_hc'] = (df_fundamentals['rel_GDP_pred_inv_emp']*
                                              df_fundamentals['hc']/df_fundamentals['hc, GBR'])

In [33]:
# I need this cell to rename the table columns
column_names = dict(zip(['rel_GDP_pred_inv','rel_GDP_pred_inv_emp','rel_GDP_pred_inv_emp_hc','rel_GDP'],
    ['Prediction, part b','Prediction, part c','Prediction, part d','Actual']))

In [34]:
df_fundamentals

,country,Investment rate,Employment growth,emp,gdpPerworker,hc,"Investment rate, GBR","Employment growth, GBR","hc, GBR","gdpPerworker, GBR",rel_GDP_pred_inv,rel_GDP,rel_GDP_pred_inv_emp,rel_GDP_pred_inv_emp_hc
countrycode,,,,,,,,,,,,,,
ABW,Aruba,0.357905,NaN,0.047601,82190.373006,NaN,0.23272,0.007309,3.773596,94307.440997,1.240131,0.871515,NaN,NaN
AGO,Angola,0.370672,0.032051,16.644962,13684.116859,1.481984,0.23272,0.007309,3.773596,94307.440997,1.262055,0.145101,1.054741,0.414223
AIA,Anguilla,0.488237,NaN,NaN,NaN,NaN,0.23272,0.007309,3.773596,94307.440997,1.448434,NaN,NaN,NaN
ALB,Albania,0.191489,-0.007132,1.075898,33282.294369,2.964992,0.23272,0.007309,3.773596,94307.440997,0.907101,0.352913,1.048813,0.824074
ARE,United Arab Emirates,0.296662,0.064468,5.808834,116760.296238,2.746695,0.23272,0.007309,3.773596,94307.440997,1.129054,1.238081,0.798882,0.581484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VNM,Viet Nam,0.211867,0.019181,50.399563,14838.496763,2.869998,0.23272,0.007309,3.773596,94307.440997,0.954147,0.157342,0.868423,0.660477
YEM,Yemen,0.124907,0.033158,5.531877,9027.142614,1.842989,0.23272,0.007309,3.773596,94307.440997,0.732617,0.095720,0.608185,0.297032
ZAF,South Africa,0.172253,0.016499,18.642710,40136.114774,2.908202,0.23272,0.007309,3.773596,94307.440997,0.860334,0.425588,0.798677,0.615517


In [37]:
# here is Solow's predicted output per worker ratios under different assumptions
results_1=(df_fundamentals[
    ['rel_GDP_pred_inv','rel_GDP_pred_inv_emp','rel_GDP_pred_inv_emp_hc','rel_GDP']].rename(columns=column_names).round(decimals=2))

In [38]:
country_list = ['ARG','BRA','CAF','DNK','NGA','ITA']

In [39]:
results_1.loc[country_list]

,"Prediction, part b","Prediction, part c","Prediction, part d",Actual
countrycode,,,,
ARG,0.83,0.75,0.62,0.51
BRA,0.92,0.87,0.71,0.35
CAF,0.70,0.64,0.27,0.03
DNK,1.05,1.08,1.03,1.15
NGA,0.96,0.80,0.42,0.14
ITA,1.06,1.09,0.91,1.04


In [ ]:
m = 10
n = 8
results_1.iloc[n*m:(n+1)*m]

In the above table, as we add more variables into our equation, Solow model's predicted income per worker ratios for Colombia is getting closer to its actual level, but not for other countries. 

In [ ]:
fig,ax = plt.subplots(figsize=(6,6))
ax.scatter(df_fundamentals['rel_GDP_pred_inv'],df_fundamentals['rel_GDP'],
          sizes=df_fundamentals['emp'].values,color=colors[0],alpha=.7,label='$\gamma$')
ax.scatter(df_fundamentals['rel_GDP_pred_inv_emp'],df_fundamentals['rel_GDP'],
          sizes=df_fundamentals['emp'].values,color=colors[1],alpha=.7,label='$\gamma,n$')
ax.scatter(df_fundamentals['rel_GDP_pred_inv_emp_hc'],df_fundamentals['rel_GDP'],
          sizes=df_fundamentals['emp'].values,color=colors[2],alpha=.7,label='$\gamma,n,h$')
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ticks = [0,.4,.8,1.2,1.6]
ax.set_xlim(-0.2,1.7)
ax.set_ylim(-0.2,1.7)
ax.set_xticks(ticks)
ax.set_yticks(ticks)


ax.legend(loc='upper left',frameon=False)
plt.savefig('./rel_GDP_pred_inv_emp_hc.svg',bbox_inches='tight')

When we look at all the countries, the model does well as we control more and more variables. However, as shown in our table, the model does not do as well with 4 examples of advanced economies. We could conclude that the Solow model does in well in accounting for the income per worker differences between developed and developing countries. But, it does not do as well for accounting for the income per worker differences between developed countries, like the UK, the US, France and Denmark listed in our question. Productivity differences is the main factor accounting the income per differences among  the developed countries.

2. Consider the extended Solow model. Suppose the production function is $Y =  K^\alpha (ehL)^{1-\alpha}.$

a. Derive change in capital per effective worker, $\dot{\tilde{k}}$, as a function of capital per effective worker, $\tilde{k}$, and other exogenous variables in the model.

b. Suppose that investment rate is 20%, $\gamma =.2$, depreciation rate is 5\%, $\delta=.05$, population growth rate is 1\%, $n=.01$, labor-augmenting technological progress rate is 2\%, $g = .02$, human capital is equal to 1, $h=1$,  and capital income share is .33, $\alpha=.33$. Find steady state capital per effective worker, income per effective worker, consumption per effective worker.

c. Suppose at time $t=0$, the economy is at the steady state, and level of labor-augmenting technology is equal to 1, $e(0)=1$. What's the income per worker level at time $t=20$? Remember that if a variable (say M) grows at a constant rate, say m, then the value of M at time $t$ is equal to $\exp(mt)$ times the value of M at time $0$, i.e. $M(t)=M(0)\exp(mt).$

d. Suppose again that at time $t=0$, the economy is at the steady state and level of labor-augmenting technology is equal to 1, $e(0)=1$. Now, suddenly (and unexpectedly) the human capital increases by 10\%, i.e. $h^{new} = 1.1$. Starting from the steady state you found in part b, simulate the model for 100 periods, and calculate capital per effective worker, capital per worker, income per effective worker, and income per worker at each time period.


In [ ]:
# First, set our parameter values
alpha = .33
delta = .05
h = 1
n = 0.01
g = 0.02
gamma = .2
# this is one way of defining a function in python
# f is our production function, output per effective workers
f = lambda k,h: k**alpha*h**(1-alpha)
# this is our kdot function
kdot = lambda k,h: gamma*k**alpha*h**(1-alpha)-(delta+n+g)*k

At the steady state:
    $$\tilde{k}^\ast = \left(\frac{\gamma}{n+g+\delta}\right)^{1/(1-\alpha)}h $$

Everytime:
  $$ \tilde{y} = \tilde{k}^\alpha h^{1-\alpha} $$
  $$ \tilde{c} = (1-\gamma)\tilde{y}$$

In [ ]:
# calculate the steady state variables
k_tilde_ss = (gamma/(n+delta+g))**(1/(1-alpha))*h
y_tilde_ss = k_tilde_ss**alpha*h**(1-alpha)
c_tilde_ss = y_tilde_ss*(1-alpha)

In [ ]:
print('Steady state capital per effective worker = ', np.round(k_tilde_ss,decimals=2))
print('Steady state ouput per effective worker = ', np.round(y_tilde_ss,decimals=2))
print('Steady state consumption per effective worker = ', np.round(c_tilde_ss,decimals=2))

Recall the definition of $\tilde{y} \equiv \frac{Y}{eL}$ and  $y\equiv \frac{Y}{L}$. Therefore, $y=e\tilde{y}$. In part c, the economy is at the steady state, $\tilde{y}\ast$, and $e$ is growing at a constant rate. But we know the initial value of $e(0)=1$, the growth rate of $e$, $g=0.02$. Therefore, $e(20)=e(0)\exp(g*20).$ Hence, $y(20)=e(20)*\tilde{y}^\ast$. 

In [ ]:
print('Income per worker at t=20 is equal to', np.round(np.exp(g*20)*y_tilde_ss,decimals=2))

To solve for part d, we first need to simulate $\tilde{k}$ and $\tilde{y}$ over time. We can quite easily calculate $e$ over time using the initial value of $e$, $e(0)$, and the growth rate of $e$, $g=0.02$. Then using $y(t)=\tilde{y(t)}e(t)$ equality, we can generete $y$ sequence over time.

In [ ]:
# k_tilde_seq will be sequence of k tilde over time
# since the economy was at the steady state, I initiate k_tilde sequence with 10 values...
# all equal to the steady state value
# you assume this is the value of k_tilde before time t=0 and at time t=0,...
# as there is no change in k_tilde at time t=0. k_tilde will begin increasing at t=1
k_tilde_seq = [k_tilde_ss,]*10

In [ ]:
# I also create a sequence of human capital
# h is equal to 1 initially, then it becomes 1.1
h_seq = np.ones(111)
# assume 9th element of the sequence corresponds to time t=0
h_seq[9:] = 1.1

In [ ]:
# starting from the 9th element, or time t=0, simulate the model to get k_tilde over time
for t in range(9,110):
    # k_prime is the next periods capital
    # k_prime is equal to current capital plus the change in capital
    k_prime = k_tilde_seq[t]+kdot(k_tilde_seq[t],h_seq[t])
    k_tilde_seq.append(k_prime)

In [ ]:
# generate e sequence as given in the formula: e(t) = e(0)*exp(g*t)
e_seq = [np.exp(t*g) for t in range(-9,102)]

In [ ]:
# k = k_tilde*e
k_seq = np.array(k_tilde_seq)*np.array(e_seq)

In [ ]:
# y_tilde = k_tilde^alpha*h^(1-alpha)
y_tilde_seq = [f(k_tilde_seq[t],h_seq[t]) for t in range(111)]

In [ ]:
# y = y_tilde*e
y_seq = np.array(y_tilde_seq)*np.array(e_seq)

In [ ]:
# now put all these variables into a table
df2 = pd.DataFrame({'Time':np.arange(-9,102),
                    'h':h_seq,
                    'k tilde':k_tilde_seq,
                   'y tilde':y_tilde_seq,
                   'e':e_seq,
                   'k':k_seq,
                   'y':y_seq})

In [ ]:
# here is how our data looks like
df2.head(15)

In [ ]:
df2[df2.Time==60].round(decimals=2)

In [ ]:
# plot k_tilde over time
fig,ax = plt.subplots()
plt.plot(df2.Time,df2['k tilde'],'k',linewidth=2)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xlabel('Time')
ax.set_title(r'$\tilde{k}$')

In [ ]:
# plot y_tilde over time
fig,ax = plt.subplots()
plt.plot(df2.Time,df2['y tilde'],'k.',linewidth=2)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xlabel('Time')
ax.set_title(r'$\tilde{y}$')

In [ ]:
# plot k over time
yticks = [5,10,20,40]
fig,ax = plt.subplots()
plt.plot(df2.Time,df2['k'],'k',linewidth=2)
plt.plot(df2.Time,k_tilde_ss*np.array(e_seq),'k--',linewidth=2)
ax.set_yscale('log')
ax.set_ylim(ymax=50)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xlabel('Time')
ax.set_title(r'$k$')

In [ ]:
# plot y over time
yticks = [2,4,8,16]
fig,ax = plt.subplots()
plt.plot(df2.Time,df2['y'],'k.',linewidth=2)
plt.plot(df2.Time,y_tilde_ss*np.array(e_seq),'k--',linewidth=2)
ax.set_yscale('log')
ax.set_ylim(ymax=20)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xlabel('Time')
ax.set_title(r'$y$')

In [ ]:
# plot growth rate of y over time
fig,ax = plt.subplots()
plt.plot(df2.Time,np.log(df2['y']).diff(),'k.',linewidth=2)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_ylim(ymax= .023)
ax.set_xlabel('Time')
ax.set_title(r'$\dot{y}/y$')